In [33]:
#!/usr/bin/env python3
"""
Adobe Hackathon Round 1B - PDF Document Intelligence System
Context-aware extraction and ranking of document sections based on persona and job-to-be-done
"""

import json
import os
import time
import re
import logging
from datetime import datetime
from typing import List, Dict, Tuple
import pickle

import pdfplumber
import nltk
import torch
import torch.nn as nn
import numpy as np
from transformers import BertTokenizer, BertModel, LayoutLMv3Processor, LayoutLMv3ForTokenClassification
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from PIL import Image

# Setup logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class BertHeadingClassifier(nn.Module):
    """BERT-based classifier for heading detection"""
    def __init__(self, bert_model):
        super(BertHeadingClassifier, self).__init__()
        self.bert = bert_model
        self.projector = nn.Linear(128, 256)
        self.classifier = nn.Sequential(
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 2)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]
        projected = self.projector(cls_embedding)
        logits = self.classifier(projected)
        return logits

class DocumentAnalyzer:
    def __init__(self):
        """Initialize the document analyzer with BERT-Tiny and LayoutLMv3 models"""
        try:
            self.bert_tokenizer = BertTokenizer.from_pretrained('./pretrained_models_bert_tiny')
            self.bert_model = BertModel.from_pretrained('./pretrained_models_bert_tiny')
            self.bert_model.eval()
            logger.info("BERT-Tiny model loaded successfully")

            self.heading_model = BertHeadingClassifier(self.bert_model)
            state_dict = torch.load('./models/document_model.pth', map_location=torch.device('cpu'))
            classifier_state = {k.replace('classifier.', ''): v for k, v in state_dict.items() if k.startswith('classifier')}
            self.heading_model.classifier.load_state_dict(classifier_state, strict=True)
            self.heading_model.eval()
            with open('./models/label_encoder.pkl', 'rb') as f:
                self.label_encoder = pickle.load(f)
            logger.info("Round 1A heading detection model loaded successfully")

            self.layoutlm_processor = LayoutLMv3Processor.from_pretrained("./models/layoutlmv3", apply_ocr=False)
            self.layoutlm_model = LayoutLMv3ForTokenClassification.from_pretrained("./models/layoutlmv3")
            self.layoutlm_model.eval()
            logger.info("LayoutLMv3 model and processor loaded successfully")

            try:
                nltk.data.find('tokenizers/punkt')
            except LookupError:
                logger.info("Downloading NLTK punkt data")
                nltk.download('punkt', quiet=True)

            try:
                nltk.data.find('taggers/averaged_perceptron_tagger_eng')
            except LookupError:
                logger.info("Downloading NLTK averaged_perceptron_tagger_eng data")
                nltk.download('averaged_perceptron_tagger_eng', quiet=True)

        except Exception as e:
            logger.error(f"Error initializing models: {str(e)}")
            raise

    def extract_domain_terms(self, persona: str, job_to_be_done: str) -> List[str]:
        """Extract up to 8 domain-specific terms"""
        combined_text = f"{persona} {job_to_be_done}".lower()
        stop_words = set(['a', 'an', 'the', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by'])
        inputs = self.bert_tokenizer(combined_text, padding=True, truncation=True, max_length=128, return_tensors='pt')
        with torch.no_grad():
            outputs = self.bert_model(**inputs)
            token_embeddings = outputs.last_hidden_state[0]
            tokens = self.bert_tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

        valid_tokens = [token for i, token in enumerate(tokens) if not token.startswith('##') and token not in stop_words and len(token) >= 3 and token not in ['[CLS]', '[SEP]']]
        valid_embeddings = [token_embeddings[i].numpy() for i, token in enumerate(tokens) if not token.startswith('##') and token not in stop_words and len(token) >= 3 and token not in ['[CLS]', '[SEP]']]

        if not valid_tokens:
            return []

        embeddings_array = np.array(valid_embeddings)
        n_clusters = min(8, len(valid_tokens))
        if n_clusters < 2:
            return valid_tokens[:8]

        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_labels = kmeans.fit_predict(embeddings_array)

        domain_terms = []
        tfidf = TfidfVectorizer().fit([combined_text])
        vocab = tfidf.vocabulary_
        tfidf_scores = tfidf.transform([combined_text]).toarray()[0]

        for cluster_id in range(n_clusters):
            cluster_indices = [i for i, label in enumerate(cluster_labels) if label == cluster_id]
            if cluster_indices:
                cluster_tokens = [valid_tokens[i] for i in cluster_indices]
                token_scores = [(token, tfidf_scores[vocab.get(token, 0)]) for token in cluster_tokens]
                token_scores.sort(key=lambda x: x[1], reverse=True)
                domain_terms.append(token_scores[0][0])

        tagged_terms = nltk.pos_tag(domain_terms)
        domain_terms = [term for term, pos in tagged_terms if pos.startswith(('NN', 'VB', 'JJ')) and len(term) >= 4]
        domain_terms = list(dict.fromkeys(domain_terms))[:8]
        if not domain_terms:
            domain_terms = valid_tokens[:8]

        logger.info(f"Extracted domain terms: {domain_terms}")
        return domain_terms

    def extract_context_keywords(self, persona: str, job_to_be_done: str) -> List[str]:
        """Extract context keywords using domain terms"""
        combined_text = f"{persona} {job_to_be_done}".lower()
        stop_words = set(['a', 'an', 'the', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by'])
        words = re.findall(r'\b[a-zA-Z]{3,}\b', combined_text)
        domain_terms = self.extract_domain_terms(persona, job_to_be_done)
        keywords = [word for word in words if word not in stop_words and (any(term in word for term in domain_terms) or len(word) >= 5)]
        keywords = list(dict.fromkeys(keywords))[:10]
        if not keywords:
            keywords = [word for word in words if word not in stop_words and len(word) >= 4][:10]
        logger.info(f"Extracted context keywords: {keywords}")
        return keywords

    def is_bullet_point(self, text: str) -> bool:
        """Detect bullet points and list-like structures"""
        bullet_patterns = [
            r'^\s*[•▪▫‣⁃\u2022]\s+', r'^\s*[-*+]\s+', r'^\s*\d+[\.\)]\s+', r'^\s*[a-zA-Z][\.\)]\s+',
            r'^\s*(tip|note|warning|caution|important):?\s+'
        ]
        return any(re.match(pattern, text, re.IGNORECASE) for pattern in bullet_patterns)

    def is_valid_heading(self, text: str) -> bool:
        """Validate heading to reject fragments and bullet points"""
        text = text.strip()
        if len(text) < 3 or len(text) > 80 or len(text.split()) < 2 or len(text.split()) > 15:
            return False
        if re.match(r'.*[:,]$', text) or text.lower().startswith(('see ', 'refer ', 'check ', 'visit ')) or self.is_bullet_point(text):
            return False
        return True

    def is_heading_rule_based(self, line: str, context_keywords: List[str], typography: Dict) -> float:
        """Rule-based heading detection with relaxed constraints"""
        line = line.strip()
        if not line or len(line) < 3 or len(line) > 80 or self.is_bullet_point(line) or not self.is_valid_heading(line):
            logger.debug(f"Line '{line}' rejected: invalid length or format")
            return 0.0

        words = line.split()
        line_lower = line.lower()
        score = 0.0

        font_size = typography.get('font_size', 12)
        avg_font_size = typography.get('avg_font_size', 12)
        is_bold = typography.get('is_bold', False)
        if font_size >= avg_font_size * 1.05:
            score += 0.35
        if is_bold:
            score += 0.25
        if line.isupper():
            score += 0.15
        if re.match(r'^[A-Z][a-z]+(\s[A-Z][a-z]+)*:?$', line):
            score += 0.25

        if 2 <= len(words) <= 10:
            score += 0.25
        elif len(words) <= 15:
            score += 0.15
        if typography.get('has_whitespace_above', False):
            score += 0.15
        if typography.get('starts_at_margin', True):
            score += 0.1

        keyword_matches = sum(1 for keyword in context_keywords if keyword in line_lower)
        score += min(0.6, keyword_matches * 0.2)
        patterns = [
            (r'^(chapter|section|part)\s+\d+', 0.3),
            (r'^(itinerary|activity|destination|sightseeing|guide|tips|experiences|cuisine|culture|adventure|attractions|nightlife)\s+', 0.4),
            (r'^(plan|explore|visit|discover|eat|stay|travel)', 0.25),
            (r'(trip|travel|culture|group|cuisine|adventure|friends)', 0.2)
        ]
        for pattern, weight in patterns:
            if re.search(pattern, line_lower):
                score += weight

        if line.endswith('.'):
            score -= 0.15
        exclusion_patterns = [
            r'^(note|tip|warning|caution|important):?\s*$',
            r'^\d+\s*(st|nd|rd|th)\s+',
            r'^(figure|table|diagram|image)\s+\d+'
        ]
        for pattern in exclusion_patterns:
            if re.match(pattern, line_lower):
                score *= 0.2

        logger.debug(f"Rule-based score for '{line}': {score:.2f}")
        return min(1.0, max(0.0, score))

    def is_heading_model_based(self, line: str) -> float:
        """Model-based heading detection"""
        try:
            inputs = self.bert_tokenizer(line, padding=True, truncation=True, max_length=128, return_tensors='pt')
            with torch.no_grad():
                logits = self.heading_model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
                probability = torch.softmax(logits, dim=1)[0, 1].item()
            logger.debug(f"BERT score for '{line}': {probability:.2f}")
            return probability
        except Exception as e:
            logger.warning(f"Model-based heading detection failed: {str(e)}")
            return 0.4

    def detect_heading(self, line: str, context_keywords: List[str], typography: Dict, layoutlm_label: str = "body") -> bool:
        """Hybrid heading detection with increased LayoutLMv3 weight"""
        rule_score = self.is_heading_rule_based(line, context_keywords, typography)
        model_score = self.is_heading_model_based(line)
        is_heading_layoutlm = layoutlm_label == "heading"
        combined_score = 0.3 * model_score + 0.3 * rule_score + 0.4 * (1.0 if is_heading_layoutlm else 0.0)
        threshold = 0.4 if any(keyword in line.lower() for keyword in context_keywords) else 0.5
        logger.debug(f"Heading detection for '{line}': rule_score={rule_score:.2f}, model_score={model_score:.2f}, layoutlm_label={layoutlm_label}, combined_score={combined_score:.2f}, threshold={threshold:.2f}")
        return combined_score > threshold

    def extract_text_from_pdf(self, pdf_path: str, context_keywords: List[str]) -> List[Dict]:
        """Extract text and sections with improved heading detection"""
        sections = []
        logger.info(f"Processing PDF: {pdf_path}")

        try:
            with pdfplumber.open(pdf_path) as pdf:
                avg_font_size = sum(char['size'] for page in pdf.pages for char in page.chars) / max(1, sum(len(page.chars) for page in pdf.pages))
                for page_num, page in enumerate(pdf.pages, 1):
                    text_lines = page.extract_text_lines(return_chars=True) or []
                    if not text_lines:
                        logger.warning(f"No text extracted from page {page_num}")
                        continue

                    page_image = page.to_image(resolution=300).original.convert("RGB")
                    words = []
                    boxes = []
                    for line in text_lines:
                        line_text = line['text'].strip()
                        if not line_text or len(line_text) < 3 or re.match(r'^\d+$', line_text):
                            continue
                        words.append(line_text)
                        x0, y0, x1, y1 = line['x0'], line['top'], line['x1'], line['bottom']
                        boxes.append([int(1000 * x0 / page.width), int(1000 * y0 / page.height), int(1000 * x1 / page.width), int(1000 * y1 / page.height)])

                    if not words:
                        continue

                    encoding = self.layoutlm_processor(page_image, words, boxes=boxes, return_tensors="pt", truncation=True, max_length=512)
                    with torch.no_grad():
                        outputs = self.layoutlm_model(**encoding)
                        logits = outputs.logits
                        predictions = torch.argmax(logits, dim=-1).squeeze().tolist()

                    id2label = {0: "body", 1: "heading"}
                    layoutlm_labels = [id2label.get(pred, "body") for pred in predictions[:len(words)]]
                    logger.debug(f"LayoutLMv3 labels for page {page_num}: {layoutlm_labels}")

                    current_section = None
                    section_text = []
                    for line_idx, (line, layoutlm_label) in enumerate(zip(text_lines, layoutlm_labels)):
                        line_text = line['text'].strip()
                        if not line_text or len(line_text) < 3 or re.match(r'^\d+$', line_text) or self.is_bullet_point(line_text):
                            continue

                        chars = line.get('chars', [])
                        font_size = max((char['size'] for char in chars), default=12) if chars else 12
                        is_bold = any(char.get('fontname', '').lower().find('bold') != -1 for char in chars)
                        has_whitespace_above = line.get('top', 0) > page.height * 0.05
                        starts_at_margin = line.get('x0', 0) < page.width * 0.05
                        typography = {
                            'font_size': font_size, 'avg_font_size': avg_font_size, 'is_bold': is_bold,
                            'has_whitespace_above': has_whitespace_above, 'starts_at_margin': starts_at_margin
                        }

                        line_text_normalized = line_text.title()
                        if self.detect_heading(line_text, context_keywords, typography, layoutlm_label):
                            if current_section and section_text:
                                combined_text = ' '.join(section_text).strip()
                                if len(combined_text.split()) >= 10:
                                    sections.append({
                                        'document': os.path.basename(pdf_path),
                                        'page': current_section['page'],
                                        'section_title': current_section['title'],
                                        'text': combined_text,
                                        'level': current_section['level']
                                    })
                            level = 1 if font_size >= avg_font_size * 1.1 else 2
                            current_section = {'title': line_text_normalized, 'page': page_num, 'level': level}
                            section_text = []
                        else:
                            if current_section:
                                section_text.append(line_text)

        except Exception as e:
            logger.error(f"Error processing {pdf_path}: {str(e)}")
            try:
                with pdfplumber.open(pdf_path) as pdf:
                    fallback_count = 0
                    for page_num, page in enumerate(pdf.pages, 1):
                        text = page.extract_text()
                        if text and len(text.strip()) > 50 and fallback_count < 5:
                            paragraphs = [p.strip() for p in text.split('\n\n') if len(p.strip()) > 20]
                            for para in paragraphs[:3]:
                                first_line = para.split('\n')[0].strip()
                                if len(first_line.split()) >= 2 and len(para.split()) >= 10 and self.is_valid_heading(first_line):
                                    chars = page.chars or []
                                    font_size = max((char['size'] for char in chars), default=12) if chars else 12
                                    if font_size >= 12:
                                        sections.append({
                                            'document': os.path.basename(pdf_path),
                                            'page': page_num,
                                            'section_title': first_line.title(),
                                            'text': para,
                                            'level': 3
                                        })
                                        fallback_count += 1
            except Exception as fallback_error:
                logger.error(f"Fallback extraction failed: {str(fallback_error)}")

        merged_sections = []
        section_map = {}
        for section in sections:
            key = (section['document'], section['section_title'].lower(), section['page'])
            if key in section_map:
                section_map[key]['text'] += ' ' + section['text']
            else:
                section_map[key] = section
        merged_sections = list(section_map.values())

        logger.info(f"Extracted {len(merged_sections)} sections from {pdf_path}")
        return merged_sections[:15]

    def encode_text_bert(self, texts: List[str], batch_size: int = 16) -> np.ndarray:
        """Encode texts with mean pooling"""
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch_texts = [re.sub(r'\s+', ' ', t.strip())[:500] for t in texts[i:i + batch_size]]
            inputs = self.bert_tokenizer(batch_texts, padding=True, truncation=True, max_length=256, return_tensors='pt')
            with torch.no_grad():
                outputs = self.bert_model(**inputs)
                attention_mask = inputs['attention_mask']
                token_embeddings = outputs.last_hidden_state
                input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
                batch_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
                embeddings.extend(batch_embeddings.numpy())
        return np.array(embeddings)

    def compute_relevance_scores(self, sections: List[Dict], persona: str, job_to_be_done: str, context_keywords: List[str]) -> List[Tuple[Dict, float]]:
        """Compute relevance scores with enhanced keyword weighting"""
        if not sections:
            return []

        logger.info(f"Computing relevance scores for {len(sections)} sections")
        keywords_str = " ".join(context_keywords)
        query = f"As a {persona}, I need to {job_to_be_done}. Focus on: {keywords_str}"
        domain_terms = self.extract_domain_terms(persona, job_to_be_done)
        section_texts = [f"{section['section_title']} {section['text']}" for section in sections]
        all_texts = [query] + section_texts
        embeddings = self.encode_text_bert(all_texts, batch_size=16)
        query_embedding = embeddings[0:1]
        section_embeddings = embeddings[1:]

        similarities = cosine_similarity(query_embedding, section_embeddings)[0]
        adjusted_scores = []
        for i, (sim, section) in enumerate(zip(similarities, sections)):
            word_count = len(section['text'].split())
            length_factor = 0.5 if word_count < 20 else min(1.5, 1 + 0.3 * np.log(word_count / 20))
            title_quality = 1.5 if any(word in section['section_title'].lower() for word in domain_terms) else 0.7 if len(section['section_title'].split()) < 2 else 1.0
            section_content = f"{section['section_title']} {section['text']}".lower()
            keyword_matches = sum(1 for keyword in context_keywords + domain_terms if keyword in section_content)
            keyword_boost = 3.0 if keyword_matches >= 3 else 2.0 if keyword_matches == 2 else 1.5 if keyword_matches == 1 else 1.0
            level_boost = 1.2 if section['level'] == 1 else 1.0 if section['level'] == 2 else 0.8
            domain_boost = 1 + sum(0.15 for term in domain_terms if term in section_content)
            adjusted_score = sim * length_factor * title_quality * keyword_boost * level_boost * domain_boost
            adjusted_scores.append(adjusted_score)

        scored_sections = list(zip(sections, adjusted_scores))
        scored_sections.sort(key=lambda x: x[1], reverse=True)
        return scored_sections[:max(7, len(scored_sections))]

    def extract_key_sentences(self, text: str, persona: str, job_to_be_done: str, context_keywords: List[str], top_k: int = 5) -> str:
        global domain_terms
        
        """Extract key sentences ensuring multi-sentence output"""
        sentences = [s.strip() for s in nltk.sent_tokenize(text) if len(s.split()) >= 15 and len(s) >= 50 and not s.lower().startswith(('note:', 'tip:', 'warning:', 'caution:')) and not s.endswith(('...', ','))]
        if len(sentences) < 2:
            paragraphs = re.split(r'\n{2,}', text)
            combined_text = ' '.join([p.strip() for p in paragraphs if len(p.strip().split()) >= 15][:3])
            sentences = [s.strip() for s in nltk.sent_tokenize(combined_text) if len(s.split()) >= 15 and len(s) >= 50 and not s.endswith(('...', ','))]
            if len(sentences) < 2:
                return combined_text[:400] if len(combined_text) > 50 else combined_text

        logger.info(f"Extracting {top_k} key sentences from {len(sentences)} sentences")
        keywords_str = " ".join(context_keywords)
        query = f"As a {persona}, I need to {job_to_be_done}. Key aspects: {keywords_str}"
        all_texts = [query] + sentences
        embeddings = self.encode_text_bert(all_texts, batch_size=16)
        query_embedding = embeddings[0:1]
        sentence_embeddings = embeddings[1:]
        similarities = cosine_similarity(query_embedding, sentence_embeddings)[0]

        adjusted_similarities = []
        for i, (sim, sentence) in enumerate(zip(similarities, sentences)):
            keyword_boost = 1 + sum(0.25 for keyword in context_keywords if keyword in sentence.lower())
            # action_boost = 0.2 if any(word in sentence.lower() for word in domain_terms) else 0.0
            adjusted_similarities.append(sim * keyword_boost * (1))

        sentence_scores = list(enumerate(adjusted_similarities))
        sentence_scores.sort(key=lambda x: x[1], reverse=True)
        top_indices = [idx for idx, _ in sentence_scores[:max(top_k, 2)]]
        top_indices.sort()
        return ' '.join(sentences[i] for i in top_indices)

def load_input_config(input_dir: str) -> Tuple[str, str]:
    """Load and validate input configuration"""
    config_path = os.path.join(input_dir, "input.json")
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"input.json not found at {config_path}")
    with open(config_path, 'r', encoding='utf-8') as f:
        config = json.load(f)
    persona = config.get('persona')
    job_to_be_done = config.get('job_to_be_done')
    if isinstance(persona, dict):
        persona = persona.get('role') or persona.get('name')
    if isinstance(job_to_be_done, dict):
        job_to_be_done = job_to_be_done.get('task') or job_to_be_done.get('description')
    if not persona or not job_to_be_done:
        raise ValueError("Both 'persona' and 'job_to_be_done' must be specified in input.json")
    return str(persona), str(job_to_be_done)

def find_pdf_files(pdfs_dir: str) -> List[str]:
    """Find and validate PDF files"""
    if not os.path.exists(pdfs_dir):
        raise FileNotFoundError(f"PDFs directory not found: {pdfs_dir}")
    pdf_files = [f for f in os.listdir(pdfs_dir) if f.lower().endswith('.pdf')]
    if not pdf_files:
        raise FileNotFoundError(f"No PDF files found in {pdfs_dir}")
    return [os.path.join(pdfs_dir, f) for f in pdf_files]

def main():
    """Main function to process documents and generate output"""
    start_time = time.time()
    try:
        input_dir = "./input"
        pdfs_dir = os.path.join(input_dir, "PDFs")
        output_dir = "./output"
        os.makedirs(output_dir, exist_ok=True)

        logger.info("Loading input configuration")
        persona, job_to_be_done = load_input_config(input_dir)
        logger.info(f"Persona: {persona}")
        logger.info(f"Job to be done: {job_to_be_done}")

        pdf_paths = find_pdf_files(pdfs_dir)
        pdf_files = [os.path.basename(path) for path in pdf_paths]
        logger.info(f"Processing {len(pdf_files)} documents: {pdf_files}")

        logger.info("Initializing document analyzer")
        analyzer = DocumentAnalyzer()

        context_keywords = analyzer.extract_context_keywords(persona, job_to_be_done)
        all_sections = []
        for pdf_path in pdf_paths:
            sections = analyzer.extract_text_from_pdf(pdf_path, context_keywords)
            all_sections.extend(sections)

        if not all_sections:
            raise ValueError("No sections extracted from any document")

        logger.info(f"Total sections extracted: {len(all_sections)}")
        scored_sections = analyzer.compute_relevance_scores(all_sections, persona, job_to_be_done, context_keywords)
        top_sections = scored_sections

        output_data = {
            "metadata": {
                "input_documents": pdf_files,
                "persona": persona,
                "job_to_be_done": job_to_be_done,
                "context_keywords": context_keywords,
                "processing_timestamp": datetime.now().isoformat(),
                "total_sections_processed": len(all_sections)
            },
            "extracted_sections": [
                {
                    "document": section['document'],
                    "section_title": section['section_title'],
                    "importance_rank": rank,
                    "page_number": section['page'],
                    "heading_level": section['level']
                } for rank, (section, _) in enumerate(top_sections, 1)
            ],
            "sub_section_analysis": [
                {
                    "document": section['document'],
                    "section_title": section['section_title'],
                    "refined_text": analyzer.extract_key_sentences(
                        section['text'], persona, job_to_be_done, context_keywords
                    ),
                    "page_number": section['page']
                } for section, _ in top_sections
            ]
        }

        output_path = os.path.join(output_dir, "output.json")
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, indent=2, ensure_ascii=False)

        processing_time = time.time() - start_time
        logger.info(f"Processing completed in {processing_time:.2f} seconds")
        logger.info(f"Output saved to {output_path}")
        for rank, (section, score) in enumerate(top_sections, 1):
            logger.info(f"{rank}. {section['document']} - {section['section_title']} (Level: {section['level']}, Score: {score:.3f})")

        return output_data

    except Exception as e:
        logger.error(f"Processing failed: {str(e)}")
        raise

if __name__ == "__main__":
    main()

#GG CODE

2025-07-27 02:53:03,391 - INFO - Loading input configuration
2025-07-27 02:53:03,394 - INFO - Persona: HR professional
2025-07-27 02:53:03,394 - INFO - Job to be done: Create and manage fillable forms for onboarding and compliance.
2025-07-27 02:53:03,396 - INFO - Processing 15 documents: ['Learn Acrobat - Create and Convert_1.pdf', 'Learn Acrobat - Create and Convert_2.pdf', 'Learn Acrobat - Edit_1.pdf', 'Learn Acrobat - Edit_2.pdf', 'Learn Acrobat - Export_1.pdf', 'Learn Acrobat - Export_2.pdf', 'Learn Acrobat - Fill and Sign.pdf', 'Learn Acrobat - Generative AI_1.pdf', 'Learn Acrobat - Generative AI_2.pdf', 'Learn Acrobat - Request e-signatures_1.pdf', 'Learn Acrobat - Request e-signatures_2.pdf', 'Learn Acrobat - Share_1.pdf', 'Learn Acrobat - Share_2.pdf', 'Test Your Acrobat Exporting Skills.pdf', 'The Ultimate PDF Sharing Checklist.pdf']
2025-07-27 02:53:03,397 - INFO - Initializing document analyzer
2025-07-27 02:53:03,456 - INFO - BERT-Tiny model loaded successfully
2025-07-27 